In [ ]:
from funcs.notebook import *

In [ ]:
# get AD tests
df = pd.read_csv("../results/adtests.csv")

# select the test results using the real data
df = df[df.dataset=="analysis"]
assert df.shape[0] == 540

In [ ]:
# check completeness: each subsample is tested on 20 different start phases
new = df.groupby(["sector",'subsample','period'])["shift"].count()
new

In [ ]:
# calculate median and std value of all 20 start phased tests
groupedmed = df.groupby(["period", "sector", "subsample"]).p.median()
groupedvar = df.groupby(["period", "sector", "subsample"]).p.std()

d = pd.DataFrame({"pval":groupedmed, "sig":groupedvar})
d

In [ ]:
# latexify table
dd = d.apply(lambda x: fr"${x.pval:.2f}$", axis=1)


ddd = dd.unstack(level=0).T

groupedn = df.groupby(["period", "sector", "subsample"]).nflares.first().unstack(level=0).T.iloc[0]
groupedn = groupedn.rename("# flares")

dddd = ddd.append(groupedn)

names = [r'$n$', "$p(P_{orb})$", "$p(P_{rot})$", "$p(P_{syn})$"]
oldnames = [r'# flares', "Orbit", "Rotation", "Beat Period"]
dddd = dddd.T.rename(index=str, columns=dict(zip(oldnames, names))).T

dddd

In [ ]:
# make it a string
stri = dddd.T[names].to_latex(index=True, escape=False, column_format=f"llcccc")
stri = stri.replace("\\toprule","\hline")
stri = stri.replace("\\midrule","\hline")
stri = stri.replace("{}","\hline")
stri = stri.replace("\\bottomrule","\hline\n" )
stri = stri.replace("%","\%" )
stri = stri.replace("high energy half","$ED>1\,$s")
stri = stri.replace("low energy half","$ED<1\,$s")
stri = stri.replace("Both Sectors","both")
stri = stri.replace("Sector 27","27")
stri = stri.replace("Sector 1","1")
stri = stri.replace("subsample &      &","sample &     $n$ &")
stri = stri.replace("period","")
stri = stri.replace("&  $n$ & $","&      & $") 
stri = stri.replace("$0.07$","$\mathbf{0.07}$") 
stri = stri.replace("sector","Sec.") 

print(stri)

In [ ]:
# save to paper
with open("/home/ekaterina/Documents/002_writing/aumic-flaring-spi-draft/tables/pvals.tex", "w") as f:
    f.write(stri)

# Other visualizations to explore

In [ ]:
f = df.groupby(["period", "sector","subsample"]).p.median()
f.unstack(level=0).unstack(level=1).plot(kind='bar', subplots=True, rot=0, 
                                         figsize=(15, 12), layout=(3, 3), legend=False,
                                         ylim=(0,1));

In [ ]:
f = df[df.period == "Orbit"].groupby(["shift","subsample"]).p.median()
f.unstack(level=0).plot(kind='bar', subplots=True, rot=0, 
                                         figsize=(15, 12), layout=(4, 5), legend=False,
                                         ylim=(0,1));

In [ ]:
f = df[df.sector=="Both Sectors"].groupby(["shift","period","subsample"]).p.median()
axes = f.unstack(level=2).unstack(level=1).plot(kind='line', subplots=True, rot=0, 
                                         figsize=(20, 20), layout=(3, 3), legend=False,
                                         ylim=(1e-3,1),xlim=(0,.9),logy=False, color="k",ylabel="p value");
[ax.legend(frameon=False) for ax_ in axes for ax in ax_];